In [ ]:
# default_exp convert

# The Converter

> The internals for the lib2nbdev functionality

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from fastcore.test import *

In [ ]:
#export
import json

from fastcore.basics import Path
from fastcore.xtras import is_listy
from fastcore.foundation import Config
from fastcore.script import call_parse

from fastprogress.fastprogress import progress_bar

from nbdev.export import nbglob, export_names, _re_class_func_def, _re_obj_def
from nbdev.sync import _split

## Foundational Helper Functions

In [ ]:
#export
def code_cell(code:str=None) -> str:
    """
    Returns a Jupyter cell with potential `code`
    """
    cell = {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
    "source": []
    }
    if is_listy(code): 
        for i, c in enumerate(code):
            if i < len(code)-1:
                cell["source"].append(c+'\n')
            else:
                cell["source"].append(c)
    elif code: cell["source"].append(code)
    return cell

A very simplistic and foundational function, it simply returns a string representation of a Jupyter cell without any metadata and potentially some code.

In [ ]:
#hide
_default_cell = "{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': []}"
test_eq(_default_cell, str(code_cell()))

In [ ]:
#export
def init_nb(module_name:str) -> str:
    """
    Initializes a complete blank notebook based on `module_name`

    Also writes the first #default_exp cell and checks for a nested module (moduleA.moduleB)
    """
    if module_name[0] == '.': module_name = module_name.split('.')[1]
    if '.ipynb' in module_name: module_name = module_name.split('.ipynb')[0]
        
    return {"cells":[code_cell(f"# default_exp {module_name}")], 
            "metadata":{
                "jupytext":{"split_at_heading":True},
                "kernelspec":{"display_name":"Python 3", "language": "python", "name": "python3"}
            },
           
           "nbformat":4,
           "nbformat_minor":4}

In [ ]:
#hide
_initial_nb = '''{
   "cells": [
      {
         "cell_type": "code",
         "execution_count": null,
         "metadata": {},
         "outputs": [],
         "source": [
            "# default_exp testname"
         ]
      }
   ],
   "metadata": {
      "jupytext": {
         "split_at_heading": true
      },
      "kernelspec": {
         "display_name": "Python 3",
         "language": "python",
         "name": "python3"
      }
   },
   "nbformat": 4,
   "nbformat_minor": 4
}'''
test_eq(_initial_nb, json.dumps(init_nb("testname"), indent=3))

In [ ]:
#export
def write_cell(code:str, is_public:bool=False) -> str:
    """
    Takes source `code`, adds an initial #export tag, and writes a Jupyter cell
    """
    if is_public is None: export = ''
    export = '#export' if is_public else '#exporti'
    source = [f"{export}"] + code.split("\n")
    return code_cell(source)

This function will write a cell given some `code` (which is a str). `is_public` is there to determine if `#export` or `#exporti` should be used (a public or private function, class, or object). 

In [ ]:
#export
def write_nb(splits:list, num:int, parent:str=None, private_list:list=[]) -> str:
    """
    Writes a fully converted Jupyter Notebook based on `splits` and saves it in `Config`'s `nbs_path`.

    The notebook number is based on `num`

    `parent` denotes if the current notebook module is based on a parent module
    such as `moduleA.moduleB`
    
    `private_list` is a by-cell list of `True`/`False` for each block of code of whether it is private or public
    """
    # Get filename
    fname = splits[0][0]
    if fname[0] == '.': fname = fname[1:]
    if parent is not None: fname = f'{parent}.{fname}'

    # Initialize and write notebook
    nb = init_nb(fname)
    for i, (_, code) in enumerate(splits):
        c = write_cell(code, private_list[i])
        nb["cells"].append(c)

    # Figure out the notebook number
    if num < 10:
        fname = f'0{num}_{fname}'
    else:
        fname = f'{num}_{fname}'

    # Save notebook in `nbs_path`
    with open(f'{Config().path("nbs_path")/fname}', 'w+') as source_nb:
        source_nb.write(json.dumps(nb))

In [ ]:
#exporti
def _not_private(n):
    "Checks if a func is private or not, alternative to nbdev's"
    for t in n.split('.'):
        if (t.startswith('_') and not t.startswith('__')): return False
    return '\\' not in t and '^' not in t and t != 'else'

## Converting Libraries

In [ ]:
#export
def generate_settings():
    """
    Guide the user for generating a proper `settings.ini` if one does not already exist in the directory    
    """
    if Path('settings.ini').exists():
        print("settings.ini already exists, please modify the existing version")
        return
    print("No settings.ini exists, let's make one:")
    f = open("settings.ini", 'w')
    user_inp = {}
    
    user_inp['[DEFAULT]\nhost'] = 'github\n'
    user_inp['lib_name'] = input("Please enter the name of your library: ")
    _enterprise = int(input("Is this an Enterprise Git? (0 or 1): "))
    
    if _enterprise:
        user_inp['repo_name'] = input("Please enter the repo name: ")
        user_inp['company_name'] = input("Please enter the company name: ")

    user_inp['user'] = input("Please enter your git username: ")
    user_inp['description'] = input("Please enter a description of the project: ")
    user_inp['keywords'] = input("Please enter some keywords for your project seperated by a space: ")
    user_inp['author'] = input("Please enter all main authors names (seperate names with a space): ")
    user_inp['author_email'] = input("Please enter a main email contact for the project: ")
    user_inp['copyright'] = input("Please enter a Copyright (such as company name or your name): ")
    user_inp['branch'] = input("Please enter the head branch of this project (such as master/main): ")
    user_inp['version'] = '0.0.1'
    user_inp['min_py'] = input("Please enter the minimum required Python for this project (such as 3.6): ")
    user_inp['audience'] = 'Developers'; user_inp['language'] = 'English'
    
    for k, v in user_inp.items(): f.write(f'{k} = {v}\n')
      
    nbs = input("Please enter where you would like your notebooks stored? (Usually `nbs` or `.`): ")

    f.write('# Set to True if you want to create a more fancy sidebar.json than the default\ncustom_sidebar = False\n')
    f.write('# Add licenses and see current list in `setup.py`\nlicense = apache2\n')
    f.write('# From 1-7: Planning Pre-Alpha Alpha Beta Production Mature Inactive\nstatus = 2\n')
    f.write('''
# Optional. Same format as setuptools requirements
# requirements = 
# Optional. Same format as setuptools console_scripts
# console_scripts = 
# Optional. Same format as setuptools dependency-links
# dep_links = ''')
    f.write('''
###
# You probably won't need to change anything under here,
#   unless you have some special requirements
###

# Change to, e.g. "nbs", to put your notebooks in nbs dir instead of repo root
''')
    f.write(f'nbs_path = {nbs}\n')
    f.write('doc_path = docs\n')
    f.write('# Whether to look for library notebooks recursively in the `nbs_path` dir\nrecursive = False\n')
    f.write("\n\n\n# Anything shown as '%(...)s' is substituted with that setting automatically")

    f.write('doc_host = https://%(user)s.github.io\n') if not _enterprise else f.write('doc_host = https://pages.github.%(company_name)s.com\n')
    f.write('doc_baseurl = /%(lib_name)s/\n') if not _enterprise else f.write('doc_baseurl  = /%(repo_name)s/%(lib_name)s/\n')
    if not _enterprise: f.write('git_url = https://github.com/%(user)s/%(lib_name)s/tree/%(branch)s/\n')
    else: f.write('git_url  = https://github.%(company_name)s.com/%(repo_name)s/%(lib_name)s/tree/%(branch)s/\n')
    f.write('\n\n\nlib_path = %(lib_name)s\ntitle = %(lib_name)s\n')
    f.write('''
#Optional advanced parameters
#Monospace docstings: adds <pre> tags around the doc strings, preserving newlines/indentation.
#monospace_docstrings = False
#Test flags: introduce here the test flags you want to use separated by |
#tst_flags = 
#Custom sidebar: customize sidebar.json yourself for advanced sidebars (False/True)
#custom_sidebar = 
#Cell spacing: if you want cell blocks in code separated by more than one new line
#cell_spacing = 
#Custom jekyll styles: if you want more jekyll styles than tip/important/warning, set them here
#jekyll_styles = note,warning,tip,important
    ''')
    print("If there are any necissary requirements for the project, please add them to the requirements section")
    print("If there should be any test flags, please add them to the tst_flags section")
    print("settings.ini successfully generated")

In [ ]:
#export
def generate_actions():
    """
    Generates a Github action for running nbdev tests
    """
    if Path('.github/workflows/main.yml'):
        print("main.yml already exists, please modify the existing version")
        return
    
    f = open(".github/workflows/main.yml", 'w')
    f.write('''
name: CI
on: [push, pull_request]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v1
    - uses: actions/setup-python@v1
      with:
        python-version: '3.6'
        architecture: 'x64'
    - name: Install the library
      run: |
        pip install nbdev jupyter
        pip install -e .
    - name: Read all notebooks
      run: |
        nbdev_read_nbs
    - name: Check if all notebooks are cleaned
      run: |
        echo "Check we are starting with clean git checkout"
        if [ -n "$(git status -uno -s)" ]; then echo "git status is not clean"; false; fi
        echo "Trying to strip out notebooks"
        nbdev_clean_nbs
        echo "Check that strip out was unnecessary"
        git status -s # display the status to see which nbs need cleaning up
        if [ -n "$(git status -uno -s)" ]; then echo -e "!!! Detected unstripped out notebooks\n!!!Remember to run nbdev_install_git_hooks"; false; fi
    - name: Check if there is no diff library/notebooks
      run: |
        if [ -n "$(nbdev_diff_nbs)" ]; then echo -e "!!! Detected difference between the notebooks and the library"; false; fi
    - name: Run tests
      run: |
        nbdev_test_nbs''')

In [ ]:
#export
@call_parse
def convert_lib():
    """
    Converts existing library to an nbdev one by autogenerating notebooks.
    
    Basic prerequisites:
      - You must have made a nbdev settings.ini file beforehand
      - Optionally you can add `# Cell` and `# Internal Cell` tags in the source files where you would like specific cells to be
    
    Run this command in the base of your repo
    
    **Can only be run once**
    """
    print('Checking for a settings.ini...')
    generate_settings()
    print('Gathering files...')
    files = nbglob(extension='.py', config_key='lib_path', recursive=True)
    if len(files) == 0: raise ValueError("No files were found, please ensure that `lib_path` is configured properly in `settings.ini`")
    print(f'{len(files)} modules found in the library')
    num_nbs = len(files)
    nb_path = Config().path('nbs_path')
    nb_path.mkdir(exist_ok=True)
    print(f'Writing notebooks to {nb_path}...')
    if nb_path.name == Config().lib_name:
        nb_path = Path('')
        slash = ''
        
    else:
        nb_path = Path(nb_path.name)
        slash = '/'

    for num, file in enumerate(progress_bar(files)):
        if (file.parent.name != Config().lib_name) and slash is not None:
            parent = file.parent.name
        else:
            parent = None
        fname = file.name.split('.py')[0] + '.ipynb'
        if fname[0] == '.': fname = fname[1:]
        # Initial string in the .py
        init_str = f"# AUTOGENERATED! DO NOT EDIT! File to edit: {nb_path}{slash}{fname} (unless otherwise specified).\n\n"

        # Override existing code to include nbdev magic and one code cell
        with open(file, encoding='utf8') as f: code = f.read()

        if "AUTOGENERATED" not in code:
            code = init_str + code

        # Check to ensure we haven't tried exporting once yet
        if "# Cell" and "# Internal Cell" not in code and '__all__' not in code:
            split_code = code.split('\n')
            private_list = []
            _do_pass, _private, _public = False, '# Internal Cell\n', '# Cell\n'
            for row, line in enumerate(split_code):
                if _do_pass: _do_pass = False; continue
                # Deal with decorators
                if '@' in line:
                    code = split_code[row+1]
                    if code[:4] == 'def ': code = code[4:]
                    if 'patch' in line or 'typedispatch' in line or not line[0].isspace():
                        is_private = _not_private(code.split('(')[0])
                        private_list.append(is_private)
                        split_code[row] = f'{_public}{line}' if is_private else f'{_private}{line}'
                    _do_pass = True
                # Deal with objects
                elif _re_obj_def.match(line) and not _do_pass:
                    is_private = _not_private(line.split('(')[0])
                    private_list.append(is_private)
                    split_code[row] = f'{_public}{line}' if is_private else f'{_private}{line}'
                # Deal with classes or functions
                elif _re_class_func_def.match(line) and not _do_pass:
                    is_private = _not_private(line.split(' ')[1].split('(')[0])
                    private_list.append(is_private)
                    split_code[row] = f'{_public}{line}' if is_private else f'{_private}{line}'
            code = '\n'.join(split_code)

        # Write to file
        with open(file, 'w', encoding='utf8') as f: f.write(code)

        # Build notebooks
        splits = _split(code)
        write_nb(splits, num, parent, private_list)
        
        # Generate the `__all__` in the top of each .py
        if '__all__' not in code:
            c = code.split("(unless otherwise specified).")
            code = c[0] + "(unless otherwise specified).\n" + f'\n__all__ = {export_names(code)}\n\n# Cell' + c[1]
            with open(file, 'w', encoding='utf8') as f: f.write(code)
    print(f"{Config().lib_name} successfully converted!")
    _workflow = int(input("Would you like to setup the automated Github workflow that nbdev provides? (0/1)"))
    if _workflow: 
        generate_actions()
        print("Github actions generated! Please make sure to include .github/actions/main.yml in your next commit!")

An example of adding in `# Cell` or `# Internal Cell` to the source code can be seen below:

```python
# Filename is noop.py

# Internal Cell
def _myPrivateFunc(o): return o

# Cell
def myPublicFunc(o): return o
```